In [ ]:
# cnn-lstm-caution-prediction模型训练代码，包含early stopping \ muti-class regression \ 多任务学习
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

# 读取数据
data = pd.read_csv('data.csv')

# 数据预处理
# 去除无效值
data = data.dropna()
# 去除异常值
data = data[(data['Caution'] >= 0) & (data['Caution'] <= 1)]
# 标准化
scaler = StandardScaler()
data[['Temperature', 'Humidity', 'Light', 'CO2', 'Ozone']] = scaler.fit_transform(data[['Temperature', 'Humidity', 'Light', 'CO2', 'Ozone']])
#数据集normalization
data['Caution'] = (data['Caution'] - data['Caution'].min()) / (data['Caution'].max() - data['Caution'].min())    # 归一化
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(data.drop('Caution', axis=1), data['Caution'], test_size=0.2, random_state=42)

# 定义cnn-lstm-caution-prediction模型，regression任务
# 定义模型
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(LSTM(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 编译模型
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

# 定义early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

# 训练模型
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# 评估模型
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('MSE: %.4f' % mse)
print('MAE: %.4f' % mae)
print('R2: %.4f' % r2)

